In [10]:
## Install packages
%pip install numpy pandas matplotlib seaborn scikit-learn torch torchvision

Note: you may need to restart the kernel to use updated packages.


In [11]:
## Import packages
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
## Load data
df = pd.read_csv('./data/food_image_df.csv')  # Replace with your actual file

## Split the DataFrame into training and validation sets
RANDSEED = 42
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=RANDSEED) # 80-20 train-validation split

## Custom Dataset Class
class FoodDataset(Dataset):
  def __init__(self, df, transform=None):
    self.df = df.reset_index(drop=True)
    self.transform = transform
    self.labels = df['label'].unique().tolist()
    self.label_to_idx = {label: idx for idx, label in enumerate(self.labels)}
  
  def __len__(self):
    return len(self.df)
  
  def __getitem__(self, idx):
    img_path = self.df.loc[idx, 'file']
    label = self.df.loc[idx, 'label']
    image = Image.open(img_path).convert('RGB')
    if self.transform:
      image = self.transform(image)
    label_idx = self.label_to_idx[label]
    return image, label_idx

## Define transforms to morph the images to same size and shape
train_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.RandomHorizontalFlip(),  # Data augmentation
  transforms.ToTensor(),
])

val_transforms = transforms.Compose([
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
])

In [13]:
## Simple CNN Class
class SimpleCNN(nn.Module):
  def __init__(self, num_classes=9):
    super(SimpleCNN, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(3, 32, kernel_size=3, padding=1),  # [batch, 32, 224, 224]
      nn.ReLU(),
      nn.MaxPool2d(2),  # [batch, 32, 112, 112]

      nn.Conv2d(32, 64, kernel_size=3, padding=1),  # [batch, 64, 112, 112]
      nn.ReLU(),
      nn.MaxPool2d(2),  # [batch, 64, 56, 56]

      nn.Conv2d(64, 128, kernel_size=3, padding=1),  # [batch, 128, 56, 56]
      nn.ReLU(),
      nn.MaxPool2d(2),  # [batch, 128, 28, 28]
    )
    self.classifier = nn.Sequential(
      nn.Linear(128 * 28 * 28, 256),
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(256, num_classes),
    )
    
  def forward(self, x):
    x = self.features(x)
    x = x.view(x.size(0), -1)  # Flatten
    x = self.classifier(x)
    return x

In [14]:
## Create train & validation datasets and dataloaders
train_dataset = FoodDataset(train_df, transform=train_transforms)
val_dataset = FoodDataset(val_df, transform=val_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

## Create model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=9).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train the model
num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward + backward + optimize
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item() * images.size(0)
  
  epoch_loss = running_loss / len(train_dataset)
  print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'FoodDataset' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


KeyboardInterrupt: 

In [15]:
## Incorporate transfer learning with a pre-trained model
from torchvision import models

## Load pretrained ResNet model
model = models.resnet18(pretrained=True)

## Freeze convolutional layers so that only the final layer is trained
for param in model.parameters():
  param.requires_grad = False

## Replace final FC layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 9)  # 9 classes

model = model.to(device)

# Optimize final layer's parameters
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/gavin/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:07<00:00, 6.04MB/s]
